In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext


from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType

In [ ]:
import numpy as np
from itertools import islice

In [2]:
from pyspark.ml.linalg import Vector, Vectors, VectorUDT,SparseVector
from pyspark.ml.feature import CountVectorizer,StopWordsRemover, HashingTF, IDF, Tokenizer

from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.feature import Word2Vec, IDF, HashingTF

In [5]:
sc = pyspark.SparkContext.getOrCreate()

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
sc.stop()

In [8]:
num_rows_to_show = 20
text_file = 'data/listings.csv'

In [9]:
sc = SparkContext() # pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc) # SparkSession.builder.appName('ops').getOrCreate()

In [12]:
df = spark.read.csv(text_file, inferSchema=True, header=True)
corpus = df.select("id", "name").dropna(subset="name")

In [13]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
docDF = tokenizer.transform(corpus)

In [82]:
Vector = CountVectorizer(inputCol="words", outputCol="vectors", minDF=2.0)
model = Vector.fit(docDF)
result = model.transform(docDF)

In [83]:
result.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(2674,[8,9,17,31,51,141,237,1189],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(2674,[20,103,2583,2615],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

# Problem

In [90]:
def my_udf_map(e):
    #print(1)
    return str(list(e.indices)) # str(type(e.values))


my_udf = udf(my_udf_map, StringType())
# my_udf = udf(my_udf_map, ArrayType(IntegerType()))

#label_udf = udf(my_udf, ArrayType(StringType()))
#                                           ^^ 
result_tmp = result.withColumn('vectors2', my_udf(result.vectors))
#df = corpus.toDF().withColumn('_3',array_to_string_udf(corpus.toDF()["_2"]))

In [93]:
result_tmp.select("id","vectors2").show(10,False)

+-----+----------------------------------+
|id   |vectors2                          |
+-----+----------------------------------+
|2818 |[8, 9, 17, 31, 51, 141, 237, 1189]|
|20168|[20, 103, 2583, 2615]             |
|25428|[1, 6, 11, 25, 32, 489]           |
|27886|[1, 22, 52, 67, 111, 134, 802]    |
|28658|[6, 8, 10, 11, 15, 301]           |
|28871|[8, 57, 105]                      |
|29051|[8, 57, 308]                      |
|31080|[0, 45, 53, 85, 1168]             |
|38266|[1, 4, 7, 42, 48, 51, 84]         |
|41125|[0, 2, 14, 243]                   |
+-----+----------------------------------+
only showing top 10 rows



In [112]:
# type(corpus.toDF()["_2"])Vectors.fromML(y)

pyspark.sql.column.Column

In [134]:
result3 = result_tmp.select("id", "vectors2").rdd.map(lambda x: [x[0], Vectors.fromJson(x[1])]).cache()

In [135]:
result3.toDF().show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3173.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3173.0 (TID 787, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-134-675b818daf1e>", line 1, in <lambda>
AttributeError: type object 'Vectors' has no attribute 'fromJson'

	at 

In [133]:
#Train the LDA model

seed=1
num_topics = 5
max_iterations = 4

model = LDA.train(result3, k=num_topics, maxIterations=max_iterations, seed=seed)

Py4JJavaError: An error occurred while calling o2284.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3172.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3172.0 (TID 786, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-131-ebcf2776b44d>", line 1, in <lambda>
  File "/opt/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/__init__.py", line 904, in fromML
 

### below working with floats approach
https://stackoverflow.com/questions/42051184/latent-dirichlet-allocation-lda-in-spark

In [117]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("./sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()



In [118]:
corpus.toDF().show(10)
# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[1.0,2.0,6.0,0.0,...|
|  1|[1.0,3.0,0.0,1.0,...|
|  2|[1.0,4.0,1.0,0.0,...|
|  3|[2.0,1.0,0.0,3.0,...|
|  4|[3.0,1.0,1.0,9.0,...|
|  5|[4.0,2.0,0.0,3.0,...|
|  6|[2.0,1.0,0.0,3.0,...|
|  7|[1.0,1.0,1.0,9.0,...|
|  8|[4.0,4.0,0.0,3.0,...|
|  9|[2.0,8.0,2.0,0.0,...|
+---+--------------------+
only showing top 10 rows

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 7.628650476326428
 4.979156269387819
 4.735714089139054
 25.91304697658815
 6.996602944717101
 2.898746009162215
 6.9812272830871995
 1.8628347265198912
 2.6050075086521263
 5.311242359191372
 17.657201268398843
Topic 1:
 9.155138563869208
 16.821502003150737
 3.7219759136356085
 5.328741466119171
 11.967598675749205
 6.21338470598839
 9.968383811448462
 5.158526010221175
 2.0990855838961497
 6.534379609537973
 9.239204319298814
Topic 2:
 9.21621095980436
 7.199341727461443
 3.542309997225338
 8.758211557292682
 6.03579837

In [169]:
tokenizer = Tokenizer(inputCol="name", outputCol="raw_words")

In [170]:
wordsData = tokenizer.transform(result) #corpus

In [171]:
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

StopWordsRemover.loadDefaultStopWords("english")

remover = StopWordsRemover(inputCol="raw_words", outputCol="words2")
wordsData = remover.transform(wordsData)

In [172]:
hashingTF = HashingTF(inputCol="words2", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

In [173]:
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=1)
idfModel = idf.fit(featurizedData)

tfidf = idfModel.transform(featurizedData)

In [183]:
tfidf.select("features").show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(10000,[494,1692,1789,2659,7293,8048,8562,9263],[2.445567235227968,2.8094747144167127,3.6888794541139363,5.2799682278798405,3.247046701834897,5.983267779903803,2.46837734783679,8.083328608786376])|
|(10000,[4235,4744,8704,9743],[7.102499355774649,4.745189363091357,2.909063890908317,8.776475789346321])                                                                                             |
|(100

In [36]:
# topics = ldaModel.topicsMatrix()
# vocabArray = model.vocabulary

In [ ]:
# wordNumbers = 10  # number of words per topic
# topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))

# def topic_render(topic):  # specify vector id of words to actual words
#     terms = topic[0]
#     result = []
#     for i in range(wordNumbers):
#         term = vocabArray[terms[i]]
#         result.append(term)
#     return result

# topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()

# for topic in range(len(topics_final)):
#     print ("Topic" + str(topic) + ":")
#     for term in topics_final[topic]:
#         print (term)
#     print ('\n')